### Базовое решение кейса "Улучшение качества видео - super resolution" 
### Кейсодержатель: RUTUBE
#### Описание решения: 
Задача Super Resolution (SR) - повышение разрешения изображений / видео с сохранением качества контента.

Приведенное базовое решение основано на алгоритмическом повышении разрешения при помощи интерполяции и улучшении качества  изображения нейронной сетью.

Однако данное решение не является единственным, существует большое количество разнообразных подходов, которые показывают лучшее качество на данной задаче. Про существующие методы решения задачи SR вы можете прочитать здесь: https://blog.paperspace.com/image-super-resolution/. 

Про baseline модель вы можете подробнее прочитать тут: https://arxiv.org/pdf/1501.00092.pdf.

![Baseline модель](SRCNN.png)

In [7]:
import os
import random
import numpy as np
import torch
from torch import nn
from torch.utils.data import Dataset
import cv2
from torchvision import transforms
import torchvision.transforms.functional as TF
from torch.optim import Adam
from torch.utils.data import DataLoader
from tqdm import tqdm

In [8]:
# фиксируем seed
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True
seed = 42
seed_everything(seed)

# Модель

In [9]:
# функция инициализации весов модели
def weights_init(m):
    if isinstance(m, nn.Conv2d):
        nn.init.kaiming_normal_(m.weight)
        if m.bias is not None:
            nn.init.constant_(m.bias, 0)
    elif isinstance(m, nn.BatchNorm2d):
        nn.init.constant_(m.weight, 1)
    elif isinstance(m, nn.Linear):
        nn.init.normal_(m.weight, 0, 0.01)
        if m.bias is not None:
            nn.init.constant_(m.bias, 0)

Статья про SRCNN - https://arxiv.org/pdf/1501.00092.pdf.

Данная архитектура не делает upsample, upsampling производится на стадии предобработки - при помощи интерполяции изображение низкого разрешения переводится в высокое разрешение, модель старается улучшить качество данного интерполированного изображения.

Модель состоит из трех сверточных слоев, для обучения используется функция потерь MSE.

In [10]:
class SRCNN(nn.Module):
    def __init__(self):
        super(SRCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=9, padding=(9 // 2))
        self.conv2 = nn.Conv2d(64, 32, kernel_size=5, padding=(5 // 2))
        self.conv3 = nn.Conv2d(32, 3, kernel_size=5, padding=(5 // 2))
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.conv1(x))
        x = self.relu(self.conv2(x))
        x = self.conv3(x)
        return x

# Датасет

Обучение модели SRCNN происходит покадрово, поэтому выберем для обучения 5000 кадров случайным образом из 1000 видео (по 5 кадров из каждого видео).

Создадим все необходимые папки, train_path - путь куда сохранятся кадры, video_path - путь к папке с исходными видео.

In [11]:
# video_path = 'path'
# train_path = './train_frames'

# lr_path = os.path.join(train_path, 'lr')
# hr_path = os.path.join(train_path, 'hr')

# if not os.path.exists(train_path):
#     os.system(f'mkdir -p {train_path}')

# if not os.path.exists(lr_path):
#     os.system(f'mkdir -p {lr_path}')

# if not os.path.exists(hr_path):
#     os.system(f'mkdir -p {hr_path}')

In [12]:
# files = os.listdir(video_path)
# pairs = []
# for f in files:
#     if f.endswith('_144.mp4'):
#         hr_name = f.split('_')[0] + '_480.mp4'
#         pairs += [(f, hr_name)]

In [13]:
# n_frames = 5000
# size = int(n_frames // len(pairs))

# save_idx = 0
# for idx in tqdm(range(len(pairs))):
#     pair = pairs[idx]

#     lr = os.path.join(video_path, pair[0])
#     hr = os.path.join(video_path, pair[1])

#     lr_cap = cv2.VideoCapture(lr)
#     hr_cap = cv2.VideoCapture(hr)

#     lr_len = int(lr_cap.get(cv2.CAP_PROP_FRAME_COUNT))
#     hr_len = int(hr_cap.get(cv2.CAP_PROP_FRAME_COUNT))

#     assert lr_len == hr_len

#     frames_idx = [i for i in range(lr_len)]
#     if size:
#         frames_idx = np.random.choice(frames_idx, size=size, replace=False)

#     tmp_idx = 0
#     while True:
#         success_lr, frame_lr = lr_cap.read()
#         success_hr, frame_hr = hr_cap.read()
#         if not success_lr or not success_hr:
#             break
#         if tmp_idx in frames_idx:
#             lr_save_path = os.path.join(lr_path, f'{save_idx}.jpg')
#             hr_save_path = os.path.join(hr_path, f'{save_idx}.jpg')
#             cv2.imwrite(lr_save_path, frame_lr)
#             cv2.imwrite(hr_save_path, frame_hr)
#             save_idx += 1
#         tmp_idx += 1

Данный класс формирует датасет для обучения / валидации и тестирования.

Структура датасета: корневая папка -> папки train / val / test -> в каждой папке train / val / test лежит 2 папки lr и hr, внутри папок лежат изображения в низком и высоком разрешениях соответственно. Названия файлов в папке lr и hr должны совпадать, например lr/frame1.jpg и hr/frame1.jpg будет использоваться как одно изображение в разных разрешениях для обучения модели.

In [14]:
class SRDataset(Dataset):
    def __init__(self, lr_path, hr_path, transform):
        self.lr = [os.path.join(lr_path, f) for f in os.listdir(lr_path)]
        self.hr = [os.path.join(hr_path, f) for f in os.listdir(hr_path)]
        self.lr, self.hr = sorted(self.lr), sorted(self.hr)
        assert len(self.lr) == len(self.hr)
        self.transform = transform

    def __len__(self):
        return len(self.lr)

    def file2np(self, path):
        img = cv2.imread(path)
        return cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    def __getitem__(self, idx):
        lr = self.file2np(self.lr[idx])
        hr = self.file2np(self.hr[idx])
        lr, hr = self.transform(lr, hr)
        return lr, hr


Аугментации ниже используются для получения torch.FloatTensor с нужными размерами.

In [15]:
# после преобразований lr и hr сохраняют пространственное соотношение
def same_transform(image1, image2, p=0.5):
    if random.random() > p:
        image1 = TF.hflip(image1)
        image2 = TF.hflip(image2)

    if random.random() > p:
        image1 = TF.vflip(image1)
        image2 = TF.vflip(image2)

    return image1, image2

In [16]:
class SameTransform(object):
    def __init__(self, hr_res, mode, crop=None):
        self.np2tensor = transforms.ToTensor()
        self.resize_lr = transforms.Resize(hr_res, antialias=None)
        self.mode = mode
        self.crop = crop

    def __call__(self, lr, hr):
        lr = self.resize_lr(self.np2tensor(lr))
        hr = self.np2tensor(hr)

        if self.mode == 'train':
            lr, hr = same_transform(lr, hr)

        if self.crop:
            i, j, h, w = transforms.RandomCrop.get_params(lr, self.crop)
            lr = TF.crop(lr, i, j, h, w)
            hr = TF.crop(hr, i, j, h, w)
            
        return lr, hr

# Обучение

In [17]:
class Trainer():
    def __init__(self):
        # устройство для обучения
        self.device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

        # количество шагов обучения
        self.n_steps = 10000

        # раз в сколько шагов выводить результаты
        self.print_interval = 25

        # раз в сколько шагов сохранять чекпоинт
        self.save_interval = 2500

        self.batch_size = 24
        self.workers = 8

        # инициализация модели
        self.srcnn = SRCNN().to(self.device)
        self.srcnn.apply(weights_init)

        # конфигурация оптимизатора Adam
        self.optimizer = Adam(
            self.srcnn.parameters(),
            0.0001
        )

        # функция потерь MSE
        self.pixel_criterion = nn.MSELoss().to(self.device)

        # разрешение hr изображения в формате (h, w)
        self.size = (480, 856)
        self.crop = (384, 384)

        # аугментации для обучения и валидации
        train_transform = SameTransform(self.size, 'train', crop=self.crop)

        # путь где хранятся папки lr и hr с изображениями
        train_prefix = './train_frames'

        # train датасет
        trainset = SRDataset(
            f'{train_prefix}/lr',
            f'{train_prefix}/hr',
            train_transform
        )

        # даталоадер для обучения батчами
        self.trainloader = DataLoader(
            trainset,
            batch_size=self.batch_size,
            shuffle=True,
            drop_last=True,
            num_workers=self.workers,
            pin_memory=True
        )

        # аугментации для инференса
        self.resize = transforms.Resize(self.size, antialias=None)
        self.np2tensor = transforms.ToTensor()

    def train_step(self, lr, hr):
        g_hr = self.srcnn(lr)
        loss = self.pixel_criterion(g_hr, hr)
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()
        return loss.item()

    def train(self):
        self.srcnn.train()
        step = 0

        while True:
            if step >= self.n_steps:
                break

            for batch in self.trainloader:
                lr, hr = batch
                lr = lr.to(self.device, non_blocking=True)
                hr = hr.to(self.device, non_blocking=True)

                mse = self.train_step(lr, hr)
                step += 1

                if step % self.print_interval == 0:
                    print(f'STEP={step} MSE={mse:.5f}')

    def frame2tensor(self, img):
        rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        hr = self.resize(self.np2tensor(rgb))
        return hr

    def tensor2frame(self, img):
        nparr = (img.detach().cpu().numpy() * 255).astype(np.uint8)
        nparr = np.transpose(nparr, (1, 2, 0))
        bgr = cv2.cvtColor(nparr, cv2.COLOR_RGB2BGR)
        return bgr

    def super_resolution(self, input_video, output_video):
        self.srcnn.eval()

        cap = cv2.VideoCapture(input_video)
        fps = cap.get(cv2.CAP_PROP_FPS)

        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        writer = cv2.VideoWriter(
            output_video,
            fourcc,
            fps,
            (self.size[1], self.size[0])
        )

        while True:
            success, frame = cap.read()
            if not success:
                break

            tensor = self.frame2tensor(frame).to(self.device)
            with torch.no_grad():
                output_tensor = self.srcnn(tensor)
            output_frame = self.tensor2frame(output_tensor)

            writer.write(output_frame)

        cap.release()
        writer.release()

In [18]:
# создаем объект - trainer для запуска процесса обучения и инференса
trainer = Trainer()

In [19]:
# запускаем процесс обучения
trainer.train()

STEP=25 MSE=0.01336
STEP=50 MSE=0.01081
STEP=75 MSE=0.00654
STEP=100 MSE=0.00375
STEP=125 MSE=0.00365
STEP=150 MSE=0.00294
STEP=175 MSE=0.00331
STEP=200 MSE=0.00240
STEP=225 MSE=0.00206
STEP=250 MSE=0.00197
STEP=275 MSE=0.00197
STEP=300 MSE=0.00160
STEP=325 MSE=0.00215
STEP=350 MSE=0.00149
STEP=375 MSE=0.00194
STEP=400 MSE=0.00171
STEP=425 MSE=0.00155
STEP=450 MSE=0.00218
STEP=475 MSE=0.00153
STEP=500 MSE=0.00165
STEP=525 MSE=0.00175
STEP=550 MSE=0.00205
STEP=575 MSE=0.00256
STEP=600 MSE=0.00157
STEP=625 MSE=0.00187
STEP=650 MSE=0.00180
STEP=675 MSE=0.00153
STEP=700 MSE=0.00161
STEP=725 MSE=0.00158
STEP=750 MSE=0.00210
STEP=775 MSE=0.00152
STEP=800 MSE=0.00159
STEP=825 MSE=0.00174
STEP=850 MSE=0.00140
STEP=875 MSE=0.00165
STEP=900 MSE=0.00153
STEP=925 MSE=0.00171
STEP=950 MSE=0.00211
STEP=975 MSE=0.00208
STEP=1000 MSE=0.00134
STEP=1025 MSE=0.00203
STEP=1050 MSE=0.00160
STEP=1075 MSE=0.00187
STEP=1100 MSE=0.00182
STEP=1125 MSE=0.00140
STEP=1150 MSE=0.00227
STEP=1175 MSE=0.00130
STEP=120

KeyboardInterrupt: 

# Инференс

Задаем путь к видео низкого разрешения, которое лежит у нас на диске (lr_video) и путь к выходному видео, обработанному моделью в высоком разрешении (hr_video).

In [ ]:
lr_video = '/home/owner/Documents/DEV/Python/SuperResolution/rutube_hackaton_super_resolution_khabarovsk/train/1_144.mp4'
hr_video = '/home/owner/Documents/DEV/Python/SuperResolution/rutube_hackaton_super_resolution_khabarovsk/train/1_480_newmp4'

trainer.super_resolution(lr_video, hr_video)